In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

audio_features_df = pd.read_csv("./preprocessing/audio_features.csv")
bert_embeddings_df = pd.read_csv("./preprocessing/bert_embeddings.csv")

# Merge datasets on Participant_ID
merged_df = pd.merge(audio_features_df, bert_embeddings_df, on="Participant_ID")

# Drop unnecessary columns
merged_df_cleaned = merged_df.drop(columns=["audio_name", "audio_class", "Label_y", "Participant_ID"])

# Extract features (X) and labels (y)
X = merged_df_cleaned.drop(columns=["Label_x"]).values  # Features
y = merged_df_cleaned["Label_x"].values  # Labels

# Standardize features (important for LSTM)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long) 

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Reshape for LSTM (batch, sequence, feature)
X_train = X_train.unsqueeze(1)  # Adding sequence dimension
X_test = X_test.unsqueeze(1)

# Define BiLSTM Model
class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiply by 2 for bidirectional

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_out = lstm_out[:, -1, :]  #
        out = self.fc(last_out)
        return out

input_dim = X_train.shape[2]  
hidden_dim = 128
output_dim = len(np.unique(y))  
num_layers = 2

model = BiLSTM(input_dim, hidden_dim, output_dim, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_classes = torch.argmax(y_pred, dim=1).numpy()

accuracy = accuracy_score(y_test.numpy(), y_pred_classes)
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test.numpy(), y_pred_classes))


Epoch [5/20], Loss: 0.3874
Epoch [10/20], Loss: 0.1089
Epoch [15/20], Loss: 0.0240
Epoch [20/20], Loss: 0.0068
Test Accuracy: 0.8889
Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.80      0.89         5

    accuracy                           0.89         9
   macro avg       0.90      0.90      0.89         9
weighted avg       0.91      0.89      0.89         9

